In [8]:
from read_delta_files_utils import load_train, load_test,create_spark_with_delta
from model_training_utils import MLExperimentManager


In [5]:
spark = create_spark_with_delta()
train = load_train(spark)
test = load_test(spark)

target_col = "in_cruz_list"  
feature_cols = [c for c in train.columns if c != target_col]

train_X = train.select(feature_cols)
train_y = train.select(target_col)
test_X = test.select(feature_cols)
test_y = test.select(target_col)

In [6]:
train_X.columns

['award_id',
 'year',
 'agency',
 'awardagencycode',
 'dirabbr',
 'divabbr',
 'fundagencycode',
 'fundprogramname',
 'orglongname',
 'orgcodedir',
 'orgcodediv',
 'orgurl',
 'perfcity',
 'perfstatecode',
 'perfcountrycode',
 'perfzipcode',
 'estimated_total_amt',
 'funds_obligated_amt',
 'startdate',
 'date',
 'expdate',
 'projectoutcomesreport',
 'title',
 'transtype',
 'formatted_at',
 'is_terminated',
 'startdate_dt',
 'representant',
 'senator']

In [9]:
manager = MLExperimentManager(
    spark=spark,
    target_col="cancelled",      # canvia-ho pel teu nom de columna
    feature_cols=None,           # o ["col1", "col2", ...]
    experiment_name="nsf_models",
    tracking_uri="file:./mlruns",
    problem_type="binary",
)

manager.load_data(load_train, load_test)

rf_model, rf_metrics = manager.train_random_forest(
    run_name="rf_baseline",
    num_trees=100,
    max_depth=10,
)

print("Mètriques validació RF:", rf_metrics)

test_metrics = manager.evaluate_on_test(rf_model)
print("Mètriques test RF:", test_metrics)

C:\Users\eloip\AppData\Roaming\Python\Python310\site-packages\mlflow\tracking\_tracking_service\utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/10 00:07:21 INFO mlflow.tracking.fluent: Experiment with name 'nsf_models' does not exist. Creating a new experiment.


Py4JJavaError: An error occurred while calling o186.fit.
: org.apache.spark.SparkException: Input column cancelled does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:134)
	at scala.collection.ArrayOps$.flatMap$extension(ArrayOps.scala:971)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:122)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:114)
	at org.apache.spark.ml.feature.StringIndexer.validateAndTransformSchema(StringIndexer.scala:152)
	at org.apache.spark.ml.feature.StringIndexer.transformSchema(StringIndexer.scala:251)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:72)
	at org.apache.spark.ml.feature.StringIndexer.fit(StringIndexer.scala:236)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorI